# Library imports, configurations

In [28]:
import os
import re
import s3fs
import pandas as pd
from unidecode import unidecode
from wordcloud import WordCloud
import nltk
nltk.download('punkt')

from stop_words import get_stop_words
import unicodedata

from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [24]:
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

In [25]:
BUCKET = "jplaton/diffusion"
fs.ls(BUCKET)

['jplaton/diffusion/.keep',
 'jplaton/diffusion/ted',
 'jplaton/diffusion/visio_mel']

# Functions

In [37]:
# Suppression des accents

def remove_accent(s):

    s = unidecode(s)
    return(s)

# Suppression de la ponctuation

def remove_punctuation(s):
    # On retire la ponctuation
 
    s = re.sub(r'[^A-Za-z]',' ',str(s))
    return(s)

# Réduction de la casse

def tolower(s):
    s = ' '.join(s.lower().split())
    return(s)


# Sur cette base, on crée une petite fonction qui retire les stop words
stopwords = get_stop_words('french')   
stopwords = [unicodedata.normalize('NFKD',m).encode('ASCII', 'ignore').decode() for m in stopwords]

def remove_stopwords(s):
    s = [w for w in word_tokenize(s) if w not in stopwords]
    s = ' '.join(s)
    return(s)

def remove_accent_from_df(df,list_cols):
    df[list_cols] = df[list_cols].apply(lambda x: remove_accent(x),axis=0)
    return 


# DATA Download

In [17]:
FILE_PATH_TED = "ted"
FILE_PATH_TED_S3 = BUCKET + "/" + FILE_PATH_TED

with fs.open(FILE_PATH_TED_S3+"/offres-datagouv-20230409.csv", mode="rb") as file_in:
    offres_2023_df = pd.read_csv(file_in, sep=";")

with fs.open(FILE_PATH_TED_S3+"/offres-datagouv-20221225.csv", mode="rb") as file_in:   
    offres_2022_df = pd.read_csv(file_in, sep=";")

with fs.open(FILE_PATH_TED_S3+"/Bulloterie_10_Saison_2.xlsx", mode="rb") as file_in:  
    bulloterie_df =  pd.read_excel(file_in, sheet_name=0,header=[0,1])

In [30]:
offres_df = pd.concat([offres_2023_df,offres_2022_df])

In [38]:
offres_df= offres_df.pipe(remove_accent_from_df,['Métier'])
offres_df

AttributeError: 'Series' object has no attribute 'encode'